## **Zero -- Shot Learning**

In [15]:
#CardiffNLP fine tuned roBERTa based one shot
#We used this model to look at hor different prompts effect the results

from transformers import pipeline

sentiment_pipeline = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")

def analyze_sentiment(tweet):
    prompt = f"Is this tweet positive, neutral or negative:\n\nTweet: \"{tweet}\"\n\nSentiment:"
    result = sentiment_pipeline(prompt)
    return result[0]['label']

tweet = "They are all like this"
sentiment = analyze_sentiment(tweet)

print(f"Sentiment: {sentiment}")


Sentiment: LABEL_1


In [16]:
#CardiffNLP fine tuned roBERTa based one shot
#We used this model to look at hor different prompts effect the results

from transformers import pipeline

sentiment_pipeline = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")

def analyze_sentiment(tweet):
    prompt = f"This looks positive. Is this tweet positive, neutral or negative:\n\nTweet: \"{tweet}\"\n\nSentiment:"
    result = sentiment_pipeline(prompt)
    return result[0]['label']

tweet = "They are all like this"
sentiment = analyze_sentiment(tweet)

print(f"Sentiment: {sentiment}")


Sentiment: LABEL_2


In [18]:
#CardiffNLP fine tuned roBERTa based one shot
#We used this model to look at hor different prompts effect the results

from transformers import pipeline

sentiment_pipeline = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")

def analyze_sentiment(tweet):
    prompt = f"This looked negative. Is this tweet positive, neutral or negative:\n\nTweet: \"{tweet}\"\n\nSentiment:"
    result = sentiment_pipeline(prompt)
    return result[0]['label']

tweet = "They are all like this"
sentiment = analyze_sentiment(tweet)

print(f"Sentiment: {sentiment}")


Sentiment: LABEL_0


In [18]:
#Roberta Based model

from transformers import pipeline


sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
tweets = [
    "I went home",
    "I love life",
    "I hate life"
]

results = sentiment_pipeline(tweets)

for tweet, result in zip(tweets, results):
    print(f"Tweet: '{tweet}' -> Sentiment: {result['label']}, Score: {result['score']:.2f}")


Tweet: 'I went home' -> Sentiment: LABEL_1, Score: 0.73
Tweet: 'I love life' -> Sentiment: LABEL_2, Score: 0.98
Tweet: 'I hate life' -> Sentiment: LABEL_0, Score: 0.97


In [19]:
#Default usage of DistilBert

from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

tweet = "I hate life"
sentiment = sentiment_pipeline(tweet)

print(f"Sentiment: {sentiment[0]['label']}, Score: {sentiment[0]['score']:.2f}")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Sentiment: NEGATIVE, Score: 1.00


In [20]:
#Distilbert based One shot
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def analyze_sentiment(tweet):
    result = sentiment_pipeline(tweet)
    return result[0]['label']

tweet = "I will wait for the new features"
sentiment = analyze_sentiment(tweet)

print(f"Sentiment: {sentiment}")


Sentiment: NEGATIVE


In [26]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

zero_shot_prompt = """
Task: Classify the sentiment of the text as either positive or negative.

Text: "I'm so not happy with the service!"
Sentiment:"""

inputs = tokenizer(zero_shot_prompt, return_tensors="pt", padding=True)
inputs["attention_mask"] = inputs["input_ids"] != tokenizer.pad_token_id
outputs = model.generate(
    inputs["input_ids"], 
    attention_mask=inputs["attention_mask"], 
    max_length=50, 
    num_return_sequences=1, 
    temperature=0.7, 
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)



Task: Classify the sentiment of the text as either positive or negative.

Text: "I'm so not happy with the service!"
Sentiment: "I'm so not happy with the service!"

Task: Create a


In [29]:
#Bart Large mnli 

from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def analyze_sentiment(tweet):
    candidate_labels = ["positive", "negative", "neutral"]
    
    result = classifier(tweet, candidate_labels)
    sentiment = result['labels'][0]  
    return sentiment

tweet = "I will go home"
sentiment = analyze_sentiment(tweet)

print(f"Sentiment: {sentiment}")


Sentiment: negative


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-large"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
zero_shot_prompt = "This looks positive. Classify the sentiment of the following text as positive or not:\nText: 'I'm so not happy with the service!'"
inputs = tokenizer(zero_shot_prompt, return_tensors="pt")

outputs = model.generate(**inputs, max_length=50)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Zero-Shot Result: {result}")


Zero-Shot Result: negative


## **Few - Shot Learning**

In [13]:
# Define the few-shot prompt with examples
few_shot_prompt = """
Classify the sentiment of the following text as positive or negative keeping in mind sarcasm as a factor.

Example 1:
Text: "I love the new update; it works perfectly!"
Sentiment: positive

Example 2:
Text: "I'm disappointed with the app; it keeps crashing."
Sentiment: negative

Example 3:
Text: "The features are good, but it could be improved."
Sentiment: neutral

Example 4:
Text: "I am so glad i have to spend all my money here. This is so not a drag"
Sentiment: negative

Now, classify the sentiment of this text:
Text: "They told me to be here. I so love being here and spending my precious time on this project"
Sentiment:
"""

inputs = tokenizer(few_shot_prompt, return_tensors="pt")

outputs = model.generate(**inputs, max_length=50)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Few-Shot Result: {result}")


Few-Shot Result: positive


In [31]:
#roBERTa Based

from transformers import pipeline

sentiment_pipeline = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")

def analyze_sentiment(tweet):
    prompt = (
        "which  sentiment of each of the following tweets as positive or negative:\n\n"
        "Example 1: 'I love this new phone; it's amazing!' -> Sentiment: positive\n"
        "Example 2: 'The weather is terrible today.' -> Sentiment: negative\n"
        "Example 3: 'Can't believe how fast it arrived, great service!' -> Sentiment: positive\n"
        f"Example 4: '{tweet}' -> Sentiment:"
    )
    result = sentiment_pipeline(prompt)
    return result[0]['label']

tweet = "I hate this app"
sentiment = analyze_sentiment(tweet)

print(f"Sentiment: {sentiment}")


Sentiment: LABEL_0
